### Assignment-2


- Create a dataframe of movies table 
- Find out the user movieID and the movie name the person has watched
- Get the movie name only for the user provided the rating after 2001
- get the total number of movies released in 1995 vs 1996
- get the number of movies released only for action genre
- get the total number of movies released for action genre
- get the genre value for each movie in a new row

In [0]:
from pyspark.sql.functions import col, from_unixtime, year, regexp_extract,split, explode

In [0]:
ratings_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/ashish258kothari@gmail.com/ratings.csv")
movies_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/ashish258kothari@gmail.com/movies.csv")


ratings_df = ratings_df.withColumn("movieId", col("movieId").cast("int"))
movies_df = movies_df.withColumn("movieId", col("movieId").cast("int"))

joined_df = ratings_df.join(movies_df, on="movieId", how="inner")


In [0]:
#Question 1
joined_df.select("userId", "movieId", "title").show(5)


+------+-------+--------------------+
|userId|movieId|               title|
+------+-------+--------------------+
|     1|      1|    Toy Story (1995)|
|     1|      3|Grumpier Old Men ...|
|     1|      6|         Heat (1995)|
|     1|     47|Seven (a.k.a. Se7...|
|     1|     50|Usual Suspects, T...|
+------+-------+--------------------+
only showing top 5 rows



In [0]:
# Question 2
ratings_df = ratings_df.withColumn("movieId", col("movieId").cast("int")) \
                       .withColumn("timestamp", col("timestamp").cast("long")) \
                       .withColumn("rating_date", from_unixtime(col("timestamp"))) \
                       .withColumn("year", year(from_unixtime(col("timestamp"))))

movies_df = movies_df.withColumn("movieId", col("movieId").cast("int"))

filtered_ratings = ratings_df.filter(col("year") > 2001)

joined_df = filtered_ratings.join(movies_df, on="movieId", how="inner")

result_df = joined_df.select("title").distinct()

result_df.show(10, truncate=False)

+--------------------------------------------------+
|title                                             |
+--------------------------------------------------+
|Tommy Boy (1995)                                  |
|Step Brothers (2008)                              |
|Dark Knight, The (2008)                           |
|Talladega Nights: The Ballad of Ricky Bobby (2006)|
|Shawshank Redemption, The (1994)                  |
|Gladiator (2000)                                  |
|Good Will Hunting (1997)                          |
|Inglourious Basterds (2009)                       |
|Departed, The (2006)                              |
|Kill Bill: Vol. 1 (2003)                          |
+--------------------------------------------------+
only showing top 10 rows



In [0]:
# Question 3

movies_df = movies_df.withColumn("release_year", regexp_extract(col("title"), r"\((\d{4})\)", 1))

filtered_df = movies_df.filter((col("release_year") == "1995") | (col("release_year") == "1996"))

filtered_df.groupBy("release_year").count().show()


+------------+-----+
|release_year|count|
+------------+-----+
|        1995|  259|
|        1996|  276|
+------------+-----+



In [0]:
# Question 4
action_movies = movies_df.filter(col("genres").like("%Action%"))

action_movies.count()

Out[16]: 1828

In [0]:
# Question 5
movies_df.filter(col("genres") == "Action").count()

Out[17]: 60

In [0]:
# Question 6
genres_exploded = movies_df.withColumn("genre", explode(split(col("genres"), "\|")))

# Optional: select only relevant columns
result_df = genres_exploded.select("movieId", "title", "genre")

# Show result
result_df.show(10, truncate=False)

+-------+-----------------------+---------+
|movieId|title                  |genre    |
+-------+-----------------------+---------+
|1      |Toy Story (1995)       |Adventure|
|1      |Toy Story (1995)       |Animation|
|1      |Toy Story (1995)       |Children |
|1      |Toy Story (1995)       |Comedy   |
|1      |Toy Story (1995)       |Fantasy  |
|2      |Jumanji (1995)         |Adventure|
|2      |Jumanji (1995)         |Children |
|2      |Jumanji (1995)         |Fantasy  |
|3      |Grumpier Old Men (1995)|Comedy   |
|3      |Grumpier Old Men (1995)|Romance  |
+-------+-----------------------+---------+
only showing top 10 rows

